In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt

import geopandas
import rasterio
import matplotlib.pyplot as plt
from shapely.geometry import Point

##### read shapefile

In [16]:
#poly = "/home/sepal-user/data/drc/RDC_Province_26.shp"
poly = "/home/sepal-user/data/drc/DRC_gadm2.shp"
# Read file using gpd.read_file()
poly_shp = gpd.read_file(poly)
poly_shp.head()

,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,COD.1.1_1,COD,Democratic Republic of the Congo,COD.1_1,Bas-Uele,NA,Aketi,NA,NA,Territoire,Territory,NA,NA,"POLYGON ((23.57258 2.39625, 23.56686 2.41616, ..."
1,COD.1.2_1,COD,Democratic Republic of the Congo,COD.1_1,Bas-Uele,NA,Aketi (ville),NA,NA,Ville,Town,NA,NA,"POLYGON ((23.85916 2.73615, 23.85623 2.71256, ..."
2,COD.1.3_1,COD,Democratic Republic of the Congo,COD.1_1,Bas-Uele,NA,Ango,NA,NA,Territoire,Territory,NA,NA,"POLYGON ((25.58740 5.36581, 25.58655 5.35884, ..."
3,COD.1.4_1,COD,Democratic Republic of the Congo,COD.1_1,Bas-Uele,NA,Bambesa,NA,NA,Territoire,Territory,NA,NA,"POLYGON ((25.25612 3.55044, 25.25667 3.55006, ..."
4,COD.1.5_1,COD,Democratic Republic of the Congo,COD.1_1,Bas-Uele,NA,Bondo,NA,NA,Territoire,Territory,NA,NA,"POLYGON ((24.47831 5.09904, 24.48372 5.09930, ..."


In [17]:
poly_crs = poly_shp.crs
print("Current CRS:", poly_crs)

Current CRS: epsg:4326


##### read points

In [28]:
esbae = pd.read_csv('/home/sepal-user/module_results/esbae/DRC/DRC_esbae_2015_2022_model.csv', delimiter=',')
esbae.columns

Index(['images', 'mon_images', 'bfast_change_date', 'bfast_magnitude',
       'bfast_means', 'cusum_change_date', 'cusum_confidence',
       'cusum_magnitude', 'red_mean', 'red_sd', 'red_min', 'red_max',
       'nir_mean', 'nir_sd', 'nir_min', 'nir_max', 'swir1_mean', 'swir1_sd',
       'swir1_min', 'swir1_max', 'swir2_mean', 'swir2_sd', 'swir2_min',
       'swir2_max', 'ndfi_mean', 'ndfi_sd', 'ndfi_min', 'ndfi_max',
       'brightness_mean', 'brightness_sd', 'brightness_min', 'brightness_max',
       'greenness_mean', 'greenness_sd', 'greenness_min', 'greenness_max',
       'wetness_mean', 'wetness_sd', 'wetness_min', 'wetness_max',
       'bs_slope_mean', 'bs_slope_sd', 'bs_slope_max', 'bs_slope_min',
       'ccdc_change_date', 'ccdc_magnitude', 'aspect', 'dw_class_mode',
       'dw_tree_prob__max', 'dw_tree_prob__min', 'dw_tree_prob__stdDev',
       'dw_tree_prob_mean', 'elevation', 'esa_lc20', 'esa_lc21', 'esri_lc17',
       'esri_lc18', 'esri_lc19', 'esri_lc20', 'esri_lc21', 'gfc_

In [32]:
esbae = esbae.drop('index_right', 'BLABLA', axis=1)

In [33]:
len(esbae)

1548365

In [34]:
gdf = geopandas.GeoDataFrame(
    esbae, geometry=geopandas.points_from_xy(esbae.LON, esbae.LAT), crs="EPSG:4326"
)

In [35]:
# Perform the spatial join
joined_data = gpd.sjoin(gdf, poly_shp, how="left", op="within",lsuffix='left', rsuffix='right')

# 'joined_data' now contains the points with additional information from the polygons

# Print or explore the resulting GeoDataFrame
joined_data.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,images,mon_images,bfast_change_date,bfast_magnitude,bfast_means,cusum_change_date,cusum_confidence,cusum_magnitude,red_mean,red_sd,...,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2
0,127,92,2016.082,-663.263855,-2.929723,2015.710,0.516891,5808.547516,9673.072153,264.749422,...,COD.20_1,Nord-Ubangi,NA,Businga,NA,NA,Territoire,Territory,NA,NA
1,121,93,2017.721,328.460571,1.608799,2016.408,0.064274,1201.461111,9954.327778,104.017425,...,COD.20_1,Nord-Ubangi,NA,Businga,NA,NA,Territoire,Territory,NA,NA
2,128,92,2015.512,-646.093933,-5.213411,2017.636,0.427163,5255.813768,9652.822283,275.581389,...,COD.20_1,Nord-Ubangi,NA,Businga,NA,NA,Territoire,Territory,NA,NA
3,127,92,2015.512,-720.501404,-8.295720,2016.258,0.105731,2415.789493,9715.193219,191.880403,...,COD.20_1,Nord-Ubangi,NA,Businga,NA,NA,Territoire,Territory,NA,NA
4,121,87,2015.534,-342.082703,-3.743933,2020.244,0.354583,3881.995676,9718.401122,240.410580,...,COD.20_1,Nord-Ubangi,NA,Businga,NA,NA,Territoire,Territory,NA,NA


In [40]:
joined_data = joined_data.drop(['NL_NAME_1', 'VARNAME_2', 'CC_2','HASC_2'])

KeyError: "['NL_NAME_1', 'VARNAME_2', 'CC_2', 'HASC_2'] not found in axis"

In [38]:
out_file =  '/home/sepal-user/module_results/esbae/DRC/DRC_esbae_2015_2022_model_terr.csv'
joined_data.to_csv(out_file,index=False)

In [3]:
esbae = pd.read_csv('/home/sepal-user/module_results/esbae/DRC/DRC_all_ceo_esbae.csv', delimiter=',', low_memory=False)
esbae.columns

Index(['images', 'mon_images', 'bfast_change_date', 'bfast_magnitude',
       'bfast_means', 'cusum_change_date', 'cusum_confidence',
       'cusum_magnitude', 'red_mean', 'red_sd',
       ...
       'Ref_ArtAg', 'Ref_IndAg', 'Ref_ArtFor', 'Ref_IndFor', 'Ref_ArtMin',
       'Ref_IndMin', 'Ref_Other', 'Source', 'Ref_Change_Year_1520',
       'Ref_Change_1520'],
      dtype='object', length=178)

In [17]:
# Get the value counts
value_counts = esbae['NAME_2'].value_counts(dropna=False)

# Convert to DataFrame
counts_df = pd.DataFrame({'NAME_2': value_counts.index, 'Count': value_counts.values})

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Display the DataFrame
print(counts_df)

                    NAME_2  Count
0               Bafwasende  41238
1                   Ubundu  34839
2                    Oshwe  34550
3                    Bondo  30921
4                  Monkoto  30311
5                  Mambasa  30051
6                     Ango  25463
7                    Dungu  24350
8                   Lomela  24004
9                    Aketi  22986
10                  Manono  22489
11                   Opala  22377
12                Shabunda  20828
13                  Dekese  20557
14             Bongandanga  20120
15                Walikale  20013
16                 Bolomba  19862
17                 Banalia  19835
18                  Kamina  18689
19                  Basoko  18599
20                  Dilolo  18264
21                   Ikela  18253
22                   Kailo  18071
23                  Yahuma  18069
24                    Poko  17849
25              Mutshatsha  17531
26                  Inongo  17234
27            Katako-Kombe  17195
28            

In [12]:
for col, dtype in zip(esbae.columns, esbae.dtypes):
    print(f'{col}: {dtype}')

images: float64
mon_images: float64
bfast_change_date: float64
bfast_magnitude: float64
bfast_means: float64
cusum_change_date: float64
cusum_confidence: float64
cusum_magnitude: float64
red_mean: float64
red_sd: float64
red_min: float64
red_max: float64
nir_mean: float64
nir_sd: float64
nir_min: float64
nir_max: float64
swir1_mean: float64
swir1_sd: float64
swir1_min: float64
swir1_max: float64
swir2_mean: float64
swir2_sd: float64
swir2_min: float64
swir2_max: float64
ndfi_mean: float64
ndfi_sd: float64
ndfi_min: float64
ndfi_max: float64
brightness_mean: float64
brightness_sd: float64
brightness_min: float64
brightness_max: float64
greenness_mean: float64
greenness_sd: float64
greenness_min: float64
greenness_max: float64
wetness_mean: float64
wetness_sd: float64
wetness_min: float64
wetness_max: float64
bs_slope_mean: float64
bs_slope_sd: float64
bs_slope_max: float64
bs_slope_min: float64
ccdc_change_date: float64
ccdc_magnitude: float64
aspect: float64
dw_class_mode: float64
dw_t

In [18]:
# Group by 'NAME_2', 'interpreted', and 'collection', then count rows
count_df = esbae.groupby(['NAME_2', 'interpreted', ']).size().reset_index(name='count')

# Pivot the table for better readability
pivot_table = count_df.pivot_table(index=['NAME_2'], columns='interpreted', values='count', fill_value=0)

# Add a column for total count
pivot_table['total'] = pivot_table.sum(axis=1)

# Display the pivot table
print(pivot_table)

interpreted                0    1  total
NAME_2                                  
Aba                       44    1     45
Aketi                  22925   61  22986
Aketi (ville)             74    2     76
Ango                   25187  276  25463
Ariwara                    1    0      1
Aru                     2883   21   2904
Aru (ville)               23    0     23
Bafwasende             41187   51  41238
Bagata                  6132   71   6203
Bambesa                 8265   28   8293
Banalia                19778   57  19835
Bandundu                  70    1     71
Bangu                      4    0      4
Basankusu              13899   38  13937
Basankusu (ville)        105    2    107
Basoko                 18547   52  18599
Basoko (ville)            25    0     25
Befale                 15041   26  15067
Bena-Dibele               26    0     26
Beni                     195    3    198
Bikoro                  9171   35   9206
Boende                 15594   39  15633
Boende (ville)  

In [20]:
out_file =  '/home/sepal-user/module_results/esbae/DRC/DRC_esbae_ceo_esbae_counts_terr.csv'
pivot_table.to_csv(out_file,index=True)